In [1]:
import imp
import Data
import utils
import Predictors, Road_Graph
imp.reload(Data)
imp.reload(utils)
imp.reload(Predictors)
imp.reload(Road_Graph)

from Road_Graph import *
from Predictors import *
from Data import *
from utils import *
import matplotlib.pyplot as plt
import numpy as np
from torch.autograd import Variable
import torch
from torch import nn, autograd
import torch.nn.functional as F

In this part, we focus on a simple road map (rep as a graph below), and generate random path on it.
<img src="../img/naive_road.png" alt="Drawing" style="width: 400px;"/>

# build the road graph as above


In [ ]:
graph_dict = {}
graph_dict = graph_dict.fromkeys(range(24))
edges = [(i, i+1) for i in range(5)] \
        + [(i+6, i+7) for i in range(5)]\
                        +[(i+12, i+13) for i in range(5)]\
                     + [(i+18, i+19) for i in range(5)]\
                     + [(0,6),(6,12), (12,18)]\
                     + [(pair[0]+1, pair[1]+1) for pair in [(0,6),(6,12), (12,18)]]\
                     + [(pair[0]+2, pair[1]+2) for pair in [(0,6),(6,12), (12,18)]]\
                     + [(pair[0]+3, pair[1]+3) for pair in [(0,6),(6,12), (12,18)]]\
                     + [(pair[0]+4, pair[1]+4) for pair in [(0,6),(6,12), (12,18)]]\
                     + [(pair[0]+5, pair[1]+5) for pair in [(0,6),(6,12), (12,18)]]
for i,j in edges:
    if not graph_dict[i]:
        graph_dict[i] = [j]
    else:
        graph_dict[i].append(j)
    if not graph_dict[j]:
        graph_dict[j] = [i]
    else:
        graph_dict[j].append(i)

#display(graph_dict)
graph = Road_Graph(graph_dict)

In [ ]:
data = random_walk_data(graph, 10000,go_back = True)

In [ ]:
mk_pd = Markov_Predictor(graph)
mk_pd.train(data)
nxt,prob = mk_pd.predict([0,1,2])
print(nxt, prob)
mk_pd.eval(random_walk_data(graph, 1000,go_back = True))

In [ ]:
graph.neighbors([0,1,2])

In [ ]:
data = random_walk_data(graph, 10000, go_back = True)

D_in, D_hidden, D_out = 24, 100, 24
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, D_hidden),
    torch.nn.ReLU(),
    torch.nn.Linear(D_hidden, D_out),
    torch.nn.Softmax(dim = 1),
)
loss_fn = torch.nn.CrossEntropyLoss()
learning_rate = 0.002
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
L = 0
n = 0
pts = {'x':[], 'y':[]}
for t in range(1,20001):
    path = data[t % len(data)]
    x,y = path_to_training_pair(path, graph)
    
    y_pred = model(x)
    loss = loss_fn(y_pred, y)
    L += sum(loss.data.numpy())
    n += len(loss.data.numpy())
    if t%1000 == 0:
        pts['x'].append(t)
        pts['y'].append(L/n)
        print(t, L/n)
        L = 0
        n = 0
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


In [ ]:
plt.plot(pts['x'],pts['y'])

In [ ]:
x,y = training_set(data[0], graph)
print(inv_one_hot(x.data.numpy()))
print(graph.neighbors(11))
print(model(x)[1][5])

In [ ]:
def comp_loss(path):
    def lable(x_raw, y):
        # x,y are road level
        nb = graph.neighbors(x_raw)
        z = y.data[0]
        return to_Variable([np.where(np.array(nb) == z)[0][0]])
    def pred(x_raw, y_):
        nb = graph.neighbors(x_raw)
        return F.softmax(torch.cat([y_[n] for n in nb]), dim=0)
    path = data[1]
    X_raw = path[:-1]
    X,Y = path_to_training_pair(path,graph)
    Y_ = model(X)
    loss = []
    for i in range(len(Y)):
        x, y, y_, x_raw = X[i], Y[i], Y_[i], X_raw[i]
        loss.append(F.cross_entropy(pred(x_raw,y_).view(1,-1),  lable(x_raw,y))) 
    return torch.sum(torch.cat(loss))

def train_step(path):
    loss = comp_loss(path)
    optimizer.zero_grad()                                                                                                                                                  
    loss.backward()                                                                                                                                                        
    optimizer.step()
def pred(x_raw):
    nb = graph.neighbors(x_raw)
    y_ = model(to_Variable(one_hot(x_raw)))
    return nb, F.softmax(torch.cat([y_[n] for n in nb]), dim=0)    

In [ ]:
path
X,Y = path_to_training_pair(path,graph)
x,y = X[0],Y[0]
inv_one_hot(x.data.numpy())

In [ ]:
x_raw = inv_one_hot(x.data.numpy())
y_ = model(x[0])
nb = graph.neighbors(x_raw)
y_ = model(to_Variable(one_hot(x_raw)))
print(nb, F.softmax(torch.cat([y_[n] for n in nb]), dim=0))

In [ ]:
def pred(x_raw):
    nb = graph.neighbors(x_raw)
    y_ = model(to_Variable(one_hot(x_raw, 24)))
    return nb, F.softmax(torch.cat([y_[n] for n in nb]), dim=0) 

In [ ]:
data = random_walk_data(graph, 10000, go_back = True)

batch_size = 1
learning_rate = 0.002 
steps = 20000
D_in, D_hidden, D_out = len(graph.nodes), 100, len(graph.nodes)                                                                                                                 

model = torch.nn.Sequential(                                                                                                                                               
    torch.nn.Linear(D_in, D_hidden),                                                                                                                                       
    torch.nn.ReLU(),                                                                                                                                                      
    torch.nn.Linear(D_hidden, D_out),
)                                                                                                                                             

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)                                                                                                         
num_batch = len(data) // batch_size

losses = []
for t in range(1,steps+1):
    i = t % num_batch
    batch_loss = Variable(torch.zeros(1), requires_grad=True)
    #num_road = 0
    y_pred_list = []
    y_list = []
    for path in data[i*batch_size:(i+1)*batch_size]:
        train_step(path)
    if t%1000 == 0:
        print(t)#, batch_loss.data[0])
    #losses.append(batch_loss.data[0])
    if i == 0:
        np.random.shuffle(data)
    #optimizer.zero_grad()                                                                                                                                                  
    #loss.backward()                                                                                                                                                        
    #optimizer.step()

In [ ]:
x,y = training_set(data[0], graph)
print(inv_one_hot(x.data.numpy()))
#print(mk_pd.predict([]))
print(graph.neighbors(5))
print(model(x)[0])

In [ ]:
#def train(self, data, batch_size = 50, learning_rate = 0.001, steps = 10000):                                                                                                  
data = random_walk_data(graph, 10000, go_back = True)

batch_size = 1
learning_rate = 0.002 
steps = 20000
D_in, D_hidden, D_out = len(graph.nodes), 100, len(graph.nodes)                                                                                                                 

model = torch.nn.Sequential(                                                                                                                                               
    torch.nn.Linear(D_in, D_hidden),                                                                                                                                       
    torch.nn.ReLU(),                                                                                                                                                      
    torch.nn.Linear(D_hidden, D_out),
    torch.nn.Softmax(dim = 1)
)                                                                                                                                             
loss_fn = torch.nn.CrossEntropyLoss()                                                                                                                                      
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)                                                                                                         
num_batch = len(data) // batch_size

losses = []
for t in range(1,steps+1):
    i = t % num_batch
    batch_loss = Variable(torch.zeros(1), requires_grad=True)
    #num_road = 0
    y_pred_list = []
    y_list = []
    for path in data[i*batch_size:(i+1)*batch_size]:
        print(i*batch_size, (i+1)*batch_size)
        x,y = training_set(path, graph)                                                                                                                                          
        y_pred = model(x)  
        loss = loss_fn(y_pred, y) # * len(y) # TODO: truancate y_pred, Contatinate
        batch_loss = batch_loss + loss
        #num_road = num_road + len(y)
        #batch_loss /= num_road
    if t%1000 == 0:
        print(t, batch_loss.data[0])
    #print(batch_loss.data[0])
    losses.append(batch_loss.data[0])
    if i == 0:
        np.random.shuffle(data)
    optimizer.zero_grad()                                                                                                                                                  
    batch_loss.backward()                                                                                                                                                        
    optimizer.step()

In [ ]:
plt.plot(losses)

In [ ]:
x,y = training_set(data[4], graph)
print(inv_one_hot(x.data.numpy()))
#print(mk_pd.predict([]))
print(graph.neighbors(13))
print(model(x)[0][19])

In [ ]:
model(x)[0,:].data[22]

In [ ]:
torch.sum(prob,1)

In [ ]:
def reweight(prob, cut):
    a,b = prob.size()
    for i in range(len(cut)):
        n = cut[i]
        if n < b:
            prob[i,n:] = 0
            prob[i, :n] = prob[i, :n] / torch.sum(prob[i, :n])
    return prob

def count_exit(path):
    return [len(graph.neighbors(i)) for i in path]

In [ ]:
prob = F.softmax(autograd.Variable(torch.randn(2, 3)), dim = 1)
cut = [1,2]
reweight(prob,cut)

In [ ]:
table = {0:{1:1, 2:4}, 1:{0:1,2:1}}
np.array(list(table[0])) / sum(list(table[0]))

In [ ]:
#def train(self, data, batch_size = 50, learning_rate = 0.001, steps = 10000):                                                                                                  
data = random_walk_data(graph, 10000,go_back = True)

batch_size = 50
learning_rate = 0.001 
steps = 1000
D_in, D_hidden, D_out = len(graph.nodes), 100, 4                                                                                                                      

model = torch.nn.Sequential(                                                                                                                                               
    torch.nn.Linear(D_in, D_hidden),                                                                                                                                       
    torch.nn.ReLU(),                                                                                                                                                       
    torch.nn.Linear(D_hidden, D_out),
    torch.nn.Softmax(dim = 1)
)

    
loss_fn = torch.nn.CrossEntropyLoss()                                                                                                                                      
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)                                                                                                         
num_batch = len(data) // batch_size

losses = []
for t in range(1,steps+1):
    if t%100 == 0:
        print(t)
    i = t % num_batch
    data_i = data[i*batch_size:(i+1)*batch_size]
    batch_loss = Variable(torch.zeros(1), requires_grad=True)
    num_road = 0
    y_pred_list = []
    y_list = []
    for path in data_i:
        num_exit = count_exit(path)[:-1]
        x,y = training_set(path, graph)                                                                                                                                          
        y_pred = reweight(model(x),num_exit)
        y_list.append(y)
        y_pred_list.append(y_pred)
        
    batch_loss = loss_fn(torch.cat(y_pred_list), torch.cat(y_list))
        #num_road = num_road + len(y)
    #batch_loss /= num_road
    losses.append(batch_loss.data[0])
    if i == 0:
        print(batch_loss.data[0])
        np.random.shuffle(data)
    optimizer.zero_grad()                                                                                                                                                  
    batch_loss.backward()                                                                                                                                                        
    optimizer.step()

In [ ]:
plt.plot(losses)

In [ ]:
path

In [ ]:
x,y = training_set(path, graph)
path = [len(graph.neighbors(inv_one_hot(r)))  for r in x.data.numpy()]
print(path)
l = [len(graph.neighbors(r))  for r in path]
y_  = model(x)
#print(y_)

def f(y_, l):
    for i in range(len(l)):
        print(y_[i,:l[i]]) #=  y_[i,:[i]] / torch.sum(y_[i,:[i]])
        print(y_[i,l[i]:]) #= 0
f(y_,l)

In [ ]:
N_batch, D_in, D_hidden, D_out = 50, 24, 100, 4
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, D_hidden),
    torch.nn.ReLU(),
    torch.nn.Linear(D_hidden, D_out),
    torch.nn.Softmax(dim = 1),
)
loss_fn = torch.nn.CrossEntropyLoss()
learning_rate = 0.002
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
L = 0
n = 0
pts = {'x':[], 'y':[]}
for t in range(1,40001):
    x,y = random_training_set()
    y_pred = model(x)
    loss = loss_fn(y_pred, y)
    L += sum(loss.data.numpy())
    n += len(loss.data.numpy())
    if t%1000 == 0:
        pts['x'].append(t)
        pts['y'].append(L/n)
        print(t, L/n)
        L = 0
        n = 0
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


In [ ]:
plt.plot(pts['x'],pts['y'])

In [ ]:
path

In [ ]:
N_batch, D_in, D_hidden, D_out = 50, 24, 100, 24
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, D_hidden),
    torch.nn.ReLU(),
    torch.nn.Linear(D_hidden, D_out),
    torch.nn.Softmax(dim = 1),
)
loss_fn = torch.nn.CrossEntropyLoss()
learning_rate = 0.002
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
L = 0
n = 0
pts = {'x':[], 'y':[]}
for t in range(1,40001):
    x,y = random_training_set_with_road_tar()
    y_pred = model(x)
    loss = loss_fn(y_pred, y)
    L += sum(loss.data.numpy())
    n += len(loss.data.numpy())
    if t%1000 == 0:
        pts['x'].append(t)
        pts['y'].append(L/n)
        print(t, L/n)
        L = 0
        n = 0
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


In [ ]:
x,y = random_training_set_with_road_tar()
print([inv_one_hot(r) for r in x.data.numpy()])
np.concatenate([model(x)[0,:].data.numpy().reshape(24,1), 
                np.array(range(0,24)).reshape(24,1)],
               axis = 1)

In [ ]:
def path_to_training_pair2(path):
    return path[1:], path[:-1]

In [ ]:
model.parameters[]

In [ ]:
x

In [ ]:
W_next.view(-1,1).repeat(1,3)

In [ ]:
model.parameters

In [ ]:
i = 0    
print('x:', x[i],'nb:', graph.neighbors(x[i]))
nb = graph.neighbors(x[i])
W_list = [W[j].view(-1,1) for j in nb]
Ws = torch.cat(W_list, dim = 1)
W_next = model(W[x[0]])
#print('W_next:', W_next)
#print(W_next.size(), Ws.size())
W_next_rep = W_next.view(-1,1).repeat(1,Ws.size()[1])
#print(W_next_rep.size(), Ws.size())
logit = torch.sum(W_next_rep*Ws, 0)
prob = F.softmax(logit, dim = 0)
print(prob)
# loss = -torch.log(prob[np.where(nb == y[i])])
# optimizer.zero_grad()
# loss.backward()
# optimizer.step()

In [ ]:
torch.log(to_Variable([1.0]))

In [ ]:
def path_to_training_pair2(path):
    return path[1:], path[:-1]

def get_prob(x):
    nb = graph.neighbors(x)
    W_list = [W[j].view(-1,1) for j in nb]
    Ws = torch.cat(W_list, dim = 1)
    W_next = model(W[x])
    #print('W_next:', W_next)
    #print(W_next.size(), Ws.size())
    W_next_rep = W_next.view(-1,1).repeat(1,Ws.size()[1])
    #print(W_next_rep.size(), Ws.size())
    logit = torch.sum(W_next_rep*Ws, 0)
    prob = F.softmax(logit, dim = 0)
    return prob

def get_loss(x, y):
    nb = graph.neighbors(x)
    #import pdb; pdb.set_trace()
    W_list = [W[j].view(-1,1) for j in nb]
    Ws = torch.cat(W_list, dim = 1)
    W_next = model(W[x])
    #print('W_next:', W_next)
    #print(W_next.size(), Ws.size())
    W_next_rep = W_next.view(-1,1).repeat(1,Ws.size()[1])
    #print(W_next_rep.size(), Ws.size())
    logit = torch.sum(W_next_rep*Ws, 0)
    prob = F.softmax(logit, dim = 0)
    loss = -torch.log(prob[np.where(nb == y)])
    return loss

In [ ]:
dtype = torch.FloatTensor

data = random_walk_data(graph,10000)

im_D = 5
N = 24
h_D = 100


loss_list = []
W = Variable(torch.randn(N,im_D).type(dtype), requires_grad=True)
model = torch.nn.Sequential(
        torch.nn.Linear(im_D, h_D),
        torch.nn.ReLU(),
        torch.nn.Linear(h_D, im_D),
    )
for k in range(len(data)):
    path = data[k]
    x,y = path_to_training_pair2(path)
    
    optimizer = torch.optim.Adam(list(model.parameters()) + [W], lr=0.001)
    for i in range(len(x)):
        loss = get_loss(x[i], y[i])
        loss_list.append(loss)
        
    if k%50 == 0:
        batch_loss = torch.sum(torch.cat(loss_list)) / len(loss_list)
        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()
        loss_list = []
        print(batch_loss.data.numpy())
 

In [ ]:
np.ones(3) /

In [ ]:
def log_loss():
    s = 0
    for i in range(24):
        p_ = get_prob(i).data.numpy()
        n = len(get_prob(i))
        s += -np.sum(np.ones(n) /n * np.log(p_))
    return s/24
log_loss()

# Esay Check 

In [ ]:
y_ = [1/3,1/3,1/3]
y = [1/3,1/3,1/3]
-np.sum(y*np.log(y_))

In [ ]:
optimizer.zero_grad()
x.backward()
optimizer.step()
x.data